In [0]:
from pyspark.sql import functions as F

# ✔️ Selecionar catálogo
spark.sql("USE CATALOG adb_cliente_savana_prd")


In [0]:
# Ler as tabelas salvas na camada Bronze
bronze_transacoes = spark.table("savana_elenir_bronze.transacoes")
bronze_clientes = spark.table("savana_elenir_bronze.clientes")

# ✔️ Visualizar para garantir que está tudo certo
display(bronze_transacoes)
display(bronze_clientes)


In [0]:
from pyspark.sql import functions as F

silver_transacoes = bronze_transacoes \
    .withColumn("id", F.regexp_replace(F.col("_id.oid"), "[-_:\\s'\\/().$!@]", "")) \
    .withColumn("oid", F.col("_id.oid")) \
    .withColumn("Meio_de_Pgmto", F.when(F.col("Meio_de_Pgmto").isNull() | (F.trim(F.col("Meio_de_Pgmto")) == ""), "não informado")
                .otherwise(F.lower(F.trim(F.col("Meio_de_Pgmto"))))) \
    .withColumn("Tipo", F.when(F.col("Tipo").isNull() | (F.trim(F.col("Tipo")) == ""), "não informado")
                .otherwise(F.lower(F.trim(F.col("Tipo"))))) \
    .drop("_id") \
    .dropna(subset=["ID_Cliente", "ID_Transacao", "Valor", "Meio_de_Pgmto", "Tipo", "id"]) \
    .dropDuplicates(["ID_Transacao", "ID_Cliente", "Valor", "Meio_de_Pgmto", "Tipo"])

# Normalização dos meios de pagamento
silver_transacoes = silver_transacoes.replace({
    "credito": "cartao de crédito",
    "crédito": "cartao de crédito",
    "debito": "cartao de débito",
    "débito": "cartao de débito"
}, subset=["Meio_de_Pgmto"])


In [0]:
# 🔹 Tratamento - CLIENTES
# ============================

cidade_correcoes = {
    "So Paulo": "São Paulo", "Joo Pessoa": "João Pessoa", "Macap": "Macapá",
    "Braslia": "Brasília", "So Luz": "São Luís", "Vitria": "Vitória"
}
nome_correcoes = {
    "Joo Souza": "João Souza", "Lcia Azevedo": "Lúcia Azevedo", "Julina Lima": "Juliana Lima",
    "Carla Per": "Carla Pereira", "Sandar Carvalho": "Sandra Carvalho",
    "Andr Barros": "André Barros", "Patrcia Gomes": "Patrícia Gomes"
}

silver_clientes = bronze_clientes \
    .withColumn("Nome", F.when(F.col("Nome").isNull() | (F.trim(F.col("Nome")) == ""), "Não informado")
                .otherwise(F.initcap(F.trim(F.col("Nome"))))) \
    .withColumn("Cidade", F.when(F.col("Cidade").isNull() | (F.trim(F.col("Cidade")) == ""), "Não informado")
                .otherwise(F.initcap(F.trim(F.col("Cidade"))))) \
    .withColumn("CPF", F.regexp_replace(F.trim(F.when(F.col("CPF").isNull(), "00000000000").otherwise(F.col("CPF"))), "[^0-9]", "")) \
    .withColumn("Agencia", F.regexp_replace(F.trim(F.when(F.col("Agencia").isNull(), "0000").otherwise(F.col("Agencia"))), "[^0-9A-Za-z]", "")) \
    .withColumn("CPF_valido", F.when(F.length(F.col("CPF")) == 11, True).otherwise(False)) \
    .replace(cidade_correcoes, subset=["Cidade"]) \
    .replace(nome_correcoes, subset=["Nome"]) \
    .fillna({"Nome": "Não informado", "Cidade": "Não informado", "CPF": "00000000000", "Agencia": "0000"}) \
    .dropDuplicates(["ID", "CPF"])




In [0]:
silver_transacoes.select("ID_Transacao").printSchema()
silver_clientes.select("Transacao").printSchema()


In [0]:
# ✔️ Remover a coluna 'id' se existir (evita conflito no join)
silver_transacoes = silver_transacoes.drop("id")

# ✔️ Realizar o JOIN entre ID_Transacao (transações) e Transacao (clientes)
transacoes_enriquecidas = silver_transacoes.join(
    silver_clientes,
    silver_transacoes["ID_Transacao"] == silver_clientes["Transacao"],
    "left"
)

# ✔️ Criar colunas de data com base na coluna dt_transacao

transacoes_enriquecidas = transacoes_enriquecidas \
    .withColumn("Ano", year(col("dt_transacao"))) \
    .withColumn("Mes", month(col("dt_transacao"))) \
    .withColumn("Dia", dayofmonth(col("dt_transacao"))) \
    .withColumn("status_data", when(col("dt_transacao") > current_date(), "Futura").otherwise("Realizada")) \
    .dropDuplicates()

# ✔️ Definir status da data
transacoes_enriquecidas = transacoes_enriquecidas \
    .withColumn("status_data", when(col("dt_transacao") > current_date(), "Futura").otherwise("Realizada")) \
    .distinct()

# ✔️ Salvar tabela enriquecida na Silver
transacoes_enriquecidas.write.mode("overwrite").format("delta") \
    .saveAsTable("savana_elenir_silver.transacoes_enriquecidas")

# ✔️ Exibir dados
display(transacoes_enriquecidas)


In [0]:
transacoes_enriquecidas.printSchema()

In [0]:
%sql
SELECT
  SUM(CASE WHEN ID_Cliente IS NULL THEN 1 ELSE 0 END) AS nulos_ID_Cliente,
  SUM(CASE WHEN ID_Transacao IS NULL THEN 1 ELSE 0 END) AS nulos_ID_Transacao,
  SUM(CASE WHEN Valor IS NULL THEN 1 ELSE 0 END) AS nulos_Valor,
  SUM(CASE WHEN Meio_de_Pgmto IS NULL THEN 1 ELSE 0 END) AS nulos_Meio_de_Pgmto,
  SUM(CASE WHEN Tipo IS NULL THEN 1 ELSE 0 END) AS nulos_Tipo,
  SUM(CASE WHEN Nome IS NULL THEN 1 ELSE 0 END) AS nulos_Nome,
  SUM(CASE WHEN Cidade IS NULL THEN 1 ELSE 0 END) AS nulos_Cidade,
  SUM(CASE WHEN dt_transacao IS NULL THEN 1 ELSE 0 END) AS nulos_dt_transacao
FROM adb_cliente_savana_prd.savana_elenir_silver.transacoes_enriquecidas;


In [0]:
%sql
SELECT *
FROM adb_cliente_savana_prd.savana_elenir_silver.transacoes_enriquecidas
WHERE ID_Cliente IS NULL
   OR ID_Transacao IS NULL
   OR Valor IS NULL
   OR Meio_de_Pgmto IS NULL
   OR Tipo IS NULL
   OR Nome IS NULL
   OR Cidade IS NULL
   OR dt_transacao IS NULL
LIMIT 50;


In [0]:
%sql
SELECT DISTINCT *
FROM adb_cliente_savana_prd.savana_elenir_silver.transacoes_enriquecidas
LIMIT 50;


In [0]:
%sql
SELECT
  COUNT(*) AS total_linhas,
  COUNT(ID_Cliente) AS nao_nulos_ID_Cliente,
  COUNT(ID_Transacao) AS nao_nulos_ID_Transacao,
  COUNT(Valor) AS nao_nulos_Valor,
  COUNT(Meio_de_Pgmto) AS nao_nulos_Meio_de_Pgmto,
  COUNT(Tipo) AS nao_nulos_Tipo,
  COUNT(Nome) AS nao_nulos_Nome,
  COUNT(Cidade) AS nao_nulos_Cidade,
  COUNT(dt_transacao) AS nao_nulos_dt_transacao
FROM adb_cliente_savana_prd.savana_elenir_silver.transacoes_enriquecidas;


In [0]:
%sql
SELECT
  COUNT(*) AS total_linhas,
  SUM(CASE WHEN ID_Cliente IS NULL THEN 1 ELSE 0 END) AS nulos_ID_Cliente,
  SUM(CASE WHEN ID_Transacao IS NULL THEN 1 ELSE 0 END) AS nulos_ID_Transacao,
  SUM(CASE WHEN Valor IS NULL THEN 1 ELSE 0 END) AS nulos_Valor,
  SUM(CASE WHEN Meio_de_Pgmto IS NULL THEN 1 ELSE 0 END) AS nulos_Meio_de_Pgmto,
  SUM(CASE WHEN Tipo IS NULL THEN 1 ELSE 0 END) AS nulos_Tipo,
  SUM(CASE WHEN Nome IS NULL THEN 1 ELSE 0 END) AS nulos_Nome,
  SUM(CASE WHEN Cidade IS NULL THEN 1 ELSE 0 END) AS nulos_Cidade,
  SUM(CASE WHEN dt_transacao IS NULL THEN 1 ELSE 0 END) AS nulos_dt_transacao
FROM adb_cliente_savana_prd.savana_elenir_silver.transacoes_enriquecidas;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW transacoes_unicas AS
SELECT DISTINCT
  ID_Transacao,
  Meio_de_Pgmto,
  Tipo,
  Valor
FROM
  adb_cliente_savana_prd.savana_elenir_bronze.transacoes;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW teste_join AS
SELECT
  c.ID             AS ID_Cliente,
  c.Nome,
  c.Transacao      AS ID_Transacao,
  c.dt_transacao,
  t.Meio_de_Pgmto,
  t.Tipo,
  t.Valor
FROM
  adb_cliente_savana_prd.savana_elenir_bronze.clientes c
INNER JOIN
  transacoes_unicas t
ON
  c.Transacao = t.ID_Transacao
WHERE
  c.dt_transacao IS NOT NULL;


In [0]:
%sql 
SELECT * FROM teste_join LIMIT 50;


In [0]:
%sql
SELECT COUNT(DISTINCT ID_Cliente) AS total_clientes
FROM teste_join;


In [0]:
%sql
CREATE OR REPLACE TABLE adb_cliente_savana_prd.savana_elenir_gold.transacoes_por_cliente_mes AS
SELECT
  ID_Cliente,
  MONTH(dt_transacao) AS Mes,
  YEAR(dt_transacao) AS Ano,
  COUNT(ID_Transacao) AS Total_Transacoes
FROM teste_join
GROUP BY ID_Cliente, MONTH(dt_transacao), YEAR(dt_transacao);



In [0]:
%sql
SELECT COUNT(*) AS total_linhas_join
FROM teste_join;


In [0]:
%sql
SELECT COUNT(DISTINCT ID_Transacao) AS total_transacoes
FROM teste_join;
